In [1]:
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple, Optional
import yfinance as yf
import scipy.stats as stats
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, ks_2samp, anderson, jarque_bera
from statsmodels.stats.diagnostic import het_arch
from statsmodels.tsa.stattools import adfuller, kpss
import warnings
import logging
import time
import cProfile
import pstats

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

warnings.filterwarnings('ignore')

class IVCrushAnalyzer:
    def __init__(self):
        self.sp500_symbols = self._get_sp500_symbols()
        self.iv_data = {}
        self.hv_data = {}
        self.options_data = {}
        self.cache = {}  # Add cache for API responses
        self.cache_timestamp = {}  # Add timestamps for cache entries
        logger.info(f"Initialized analyzer with {len(self.sp500_symbols)} symbols")

    def _get_sp500_symbols(self) -> List[str]:
        """Fetch S&P 500 symbols using yfinance"""
        try:
            sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
            return sp500['Symbol'].tolist()
        except Exception as e:
            logger.error(f"Error fetching S&P 500 symbols: {e}")
            return []

    def fetch_options_data(self, symbol: str, timeout: int = 5, cache_expiry: int = 300) -> Dict:
        """Fetch options chain data for a symbol with timeout and caching"""
        cache_key = f"{symbol}_options"
        current_time = time.time()

        # Check cache with expiry
        if cache_key in self.cache:
            cache_age = current_time - self.cache_timestamp.get(cache_key, 0)
            if cache_age < cache_expiry:
                logger.info(f"Using cached data for {symbol} (age: {cache_age:.1f}s)")
                return self.cache[cache_key]
            else:
                logger.info(f"Cache expired for {symbol}")

        try:
            start_time = current_time
            logger.info(f"Fetching data for {symbol}")
            stock = yf.Ticker(symbol)
            options_data = {}

            for expiry in stock.options[:4]:  # Get first 4 expiration dates
                if time.time() - start_time > timeout:
                    logger.warning(f"Timeout fetching options data for {symbol}")
                    break

                try:
                    logger.info(f"Fetching {expiry} options for {symbol}")
                    opt = stock.option_chain(expiry)
                    if opt.calls.empty or opt.puts.empty:
                        logger.warning(f"Empty options data for {symbol} expiry {expiry}")
                        continue

                    options_data[expiry] = {
                        'calls': opt.calls,
                        'puts': opt.puts,
                        'expiry': expiry
                    }
                    logger.info(f"Successfully fetched {expiry} options for {symbol}")
                except Exception as e:
                    logger.error(f"Error fetching options data for {symbol} expiry {expiry}: {e}")
                    continue

            if options_data:
                self.cache[cache_key] = options_data
                self.cache_timestamp[cache_key] = current_time
                logger.info(f"Cached data for {symbol}")
            return options_data

        except Exception as e:
            logger.error(f"Error in fetch_options_data for {symbol}: {e}")
            return {}

    def calculate_iv_crush_probability(self, symbol: str, timeout: int = 15) -> Dict:
        """Calculate probability of IV crush based on all indicators"""
        logger.info(f"\nAnalyzing {symbol}...")
        start_time = time.time()

        try:
            # Fetch options data with timeout
            if time.time() - start_time > timeout:
                logger.warning(f"Timeout during analysis of {symbol}")
                return {}

            options_data = self.fetch_options_data(symbol)
            if not options_data:
                logger.warning(f"No options data available for {symbol}")
                return {}

            # Quick validation of required data
            if len(options_data) < 2:
                logger.warning(f"Insufficient options data for {symbol}")
                return {}

            iv_term = self.calculate_term_structure(options_data)
            if len(iv_term) < 2:
                logger.warning(f"Insufficient term structure data for {symbol}")
                return {}

            front_back_ratio = iv_term[min(iv_term.keys())] / iv_term[max(iv_term.keys())]

            # Check timeout before heavy computations
            if time.time() - start_time > timeout:
                logger.warning(f"Timeout during analysis of {symbol}")
                return {}

            iv_hv_data = self.analyze_iv_hv_spread(symbol)
            volume_data = self.analyze_volume_signals(options_data)
            spread_data = self.analyze_bid_ask_spreads(options_data)

            detail_scores = {
                'term_structure_score': self._score_term_structure(front_back_ratio),
                'iv_hv_score': self._score_iv_hv_spread(iv_hv_data.get('iv_hv_ratio', 0)),
                'volume_score': self._score_volume_patterns(volume_data),
                'spread_score': self._score_bid_ask_patterns(spread_data)
            }

            weights = {
                'term_structure_score': 0.35,
                'iv_hv_score': 0.30,
                'volume_score': 0.20,
                'spread_score': 0.15
            }

            total_probability = sum(detail_scores[k] * weights[k] for k in detail_scores.keys())

            result = {
                'symbol': symbol,
                'iv_crush_probability': total_probability,
                'detail_scores': detail_scores,
                'raw_metrics': {
                    'front_back_ratio': front_back_ratio,
                    'iv_hv_ratio': iv_hv_data.get('iv_hv_ratio', 0),
                    'volume_ratio': volume_data.get('volume_ratio', 0),
                    'spread_ratio': max(spread_data.values()) / min(spread_data.values()) if spread_data else 0
                }
            }
            logger.info(f"Successfully analyzed {symbol} in {time.time() - start_time:.2f} seconds")
            return result

        except Exception as e:
            logger.error(f"Error calculating IV crush probability for {symbol}: {e}")
            return {}

    def calculate_term_structure(self, options_data: Dict) -> Dict[str, float]:
        """Calculate IV term structure metrics"""
        if not options_data:
            logger.warning("No options data available for term structure calculation")
            return {}

        expiries = sorted(options_data.keys())
        if len(expiries) < 2:
            logger.warning("Insufficient expiries for term structure calculation")
            return {}

        iv_term = {}
        for expiry in expiries:
            try:
                calls = options_data[expiry]['calls']
                puts = options_data[expiry]['puts']

                weight = (calls['volume'] * calls['openInterest']).fillna(0)
                weighted_iv = (calls['impliedVolatility'] * weight).sum() / weight.sum() if weight.sum() > 0 else 0

                iv_term[expiry] = weighted_iv
            except Exception as e:
                logger.error(f"Error calculating term structure for expiry {expiry}: {e}")
                continue

        return iv_term

    def analyze_iv_hv_spread(self, symbol: str) -> Dict[str, float]:
        """Analyze spread between IV and historical volatility"""
        try:
            stock = yf.Ticker(symbol)

            hist_data = stock.history(period='1y')
            if len(hist_data) < 20:
                return {}

            returns = np.log(hist_data['Close'] / hist_data['Close'].shift(1))
            hv_20 = returns.rolling(window=20).std() * np.sqrt(252)
            hv_current = hv_20.iloc[-1]

            options = self.fetch_options_data(symbol)
            if not options:
                return {}

            front_month_iv = self.calculate_term_structure(options).get(min(options.keys()))
            if not front_month_iv:
                return {}

            return {
                'hv': hv_current,
                'iv': front_month_iv,
                'iv_hv_ratio': front_month_iv / hv_current if hv_current > 0 else 0
            }
        except Exception as e:
            logger.error(f"Error analyzing IV-HV spread for {symbol}: {e}")
            return {}

    def analyze_volume_signals(self, options_data: Dict) -> Dict[str, float]:
        """Analyze options volume and open interest patterns"""
        if not options_data:
            return {'volume_ratio': 0, 'oi_ratio': 0}

        try:
            near_term = min(options_data.keys())
            far_term = max(options_data.keys())

            near_calls = options_data[near_term]['calls']
            near_puts = options_data[near_term]['puts']
            far_calls = options_data[far_term]['calls']
            far_puts = options_data[far_term]['puts']

            near_volume = (near_calls['volume'].sum() + near_puts['volume'].sum())
            far_volume = (far_calls['volume'].sum() + far_puts['volume'].sum())
            volume_ratio = near_volume / far_volume if far_volume > 0 else 0

            near_oi = (near_calls['openInterest'].sum() + near_puts['openInterest'].sum())
            far_oi = (far_calls['openInterest'].sum() + far_puts['openInterest'].sum())
            oi_ratio = near_oi / far_oi if far_oi > 0 else 0

            return {
                'volume_ratio': volume_ratio,
                'oi_ratio': oi_ratio
            }
        except Exception as e:
            logger.error(f"Error analyzing volume signals: {e}")
            return {'volume_ratio': 0, 'oi_ratio': 0}


    def analyze_bid_ask_spreads(self, options_data: Dict) -> Dict[str, float]:
        """Analyze bid-ask spread patterns"""
        if not options_data:
            return {}

        spreads = {}
        for expiry, data in options_data.items():
            calls = data['calls']
            puts = data['puts']

            call_spreads = (calls['ask'] - calls['bid']) / ((calls['ask'] + calls['bid']) / 2)
            put_spreads = (puts['ask'] - puts['bid']) / ((puts['ask'] + puts['bid']) / 2)

            spreads[expiry] = (call_spreads.mean() + put_spreads.mean()) / 2

        return spreads

    def _score_term_structure(self, ratio: float) -> float:
        """Score term structure steepness"""
        if ratio > 1.3:
            return 0.8
        elif ratio > 1.2:
            return 0.6
        elif ratio > 1.1:
            return 0.4
        return 0.2

    def _score_iv_hv_spread(self, ratio: float) -> float:
        """Score IV-HV spread"""
        if ratio > 2.0:
            return 0.9
        elif ratio > 1.5:
            return 0.7
        elif ratio > 1.2:
            return 0.5
        return 0.3

    def _score_volume_patterns(self, volume_data: Dict) -> float:
        """Score volume and OI patterns"""
        volume_ratio = volume_data.get('volume_ratio', 0)
        oi_ratio = volume_data.get('oi_ratio', 0)

        if volume_ratio > 3.0 and oi_ratio > 2.0:
            return 0.9
        elif volume_ratio > 2.0 and oi_ratio > 1.5:
            return 0.7
        elif volume_ratio > 1.5 and oi_ratio > 1.2:
            return 0.5
        return 0.3

    def _score_bid_ask_patterns(self, spread_data: Dict) -> float:
        """Score bid-ask spread patterns"""
        if not spread_data:
            return 0

        spread_ratio = max(spread_data.values()) / min(spread_data.values())

        if spread_ratio > 2.0:
            return 0.8
        elif spread_ratio > 1.5:
            return 0.6
        elif spread_ratio > 1.2:
            return 0.4
        return 0.2

    def visualize_results(self, results: List[Dict]):
        """Visualize analysis results"""
        if not results:
            logger.warning("No results to visualize")
            return

        logger.info(f"\nVisualizing results for {len(results)} symbols...")

        try:
            # Convert results to DataFrame
            df = pd.DataFrame(results)

            # First plot: Probability Distribution
            logger.info("Creating probability distribution plot...")
            plt.figure(figsize=(12, 6))
            if 'iv_crush_probability' in df.columns:
                sns.kdeplot(data=df, x='iv_crush_probability', fill=True)
                plt.title('IV Crush Probability Distribution')
                plt.xlabel('Probability')
                plt.ylabel('Density')
                plt.grid(True, alpha=0.3)
                plt.savefig('probability_distribution.png')
                plt.close()
            else:
                logger.warning("No probability data available for visualization")

            # Second plot: Component Scores
            logger.info("Creating component scores plot...")
            plt.figure(figsize=(15, 8))

            # Extract scores safely
            scores_data = []
            for idx, row in df.iterrows():
                if isinstance(row.get('detail_scores'), dict):
                    score_dict = {
                        'symbol': row['symbol'],
                    }
                    # Safely extract each score component
                    if isinstance(row.get('detail_scores'), dict):
                        for key, value in row['detail_scores'].items():
                            score_dict[key] = value
                    scores_data.append(score_dict)

            if scores_data:
                scores_df = pd.DataFrame(scores_data)
                if not scores_df.empty and len(scores_df.columns) > 1:  # Ensure we have more than just the symbol column
                    scores_df.set_index('symbol', inplace=True)

                    # Create heatmap
                    sns.heatmap(scores_df, annot=True, cmap='YlOrRd', center=0.5)
                    plt.title('Component Scores Comparison')
                    plt.xlabel('Score Components')
                    plt.ylabel('Symbols')
                    plt.xticks(rotation=45)
                    plt.tight_layout()
                    plt.savefig('component_scores.png')
                    plt.close()
                else:
                    logger.warning("Insufficient data for component scores visualization")
            else:
                logger.warning("No score data available for visualization")

            # Display text results
            logger.info("\nAnalysis Results Summary:")
            summary_df = df[['symbol', 'iv_crush_probability']].sort_values('iv_crush_probability', ascending=False)
            logger.info("\nTop IV Crush Candidates:")
            logger.info("\n" + str(summary_df))

        except Exception as e:
            logger.error(f"Error in visualization: {e}")
            logger.exception("Full visualization error traceback:")



class IVCrushAnalyzerEnhanced(IVCrushAnalyzer):
    def __init__(self):
        super().__init__()
        self.historical_iv = {}
        self.statistical_tests = {}
        self.profiling_enabled = False
        self.profile = None

    def enable_profiling(self):
        """Enable performance profiling"""
        self.profiling_enabled = True
        self.profile = cProfile.Profile()

    def disable_profiling(self):
        """Disable performance profiling and print stats"""
        if self.profile:
            self.profile.disable()
            stats = pstats.Stats(self.profile)
            stats.sort_stats('cumulative')
            stats.print_stats()
        self.profiling_enabled = False

    def _calculate_evt_metrics(self, series: np.array) -> Dict[str, float]:
        """Calculate EVT metrics with enhanced error handling and normalization"""
        try:
            # Handle edge cases
            if len(series) < 2 or np.all(np.isnan(series)):
                logger.warning("Insufficient data for EVT calculation")
                return {
                    'var_95': 0,
                    'var_99': 0,
                    'es_95': 0,
                    'es_99': 0
                }

            # Remove NaN values
            series = series[~np.isnan(series)]

            # Normalize returns for consistent scaling
            normalized_series = (series - np.mean(series)) / np.std(series)

            # Calculate VaR with error handling
            var_95 = np.percentile(normalized_series, 5)
            var_99 = np.percentile(normalized_series, 1)

            # Calculate ES with error handling
            es_95 = np.mean(normalized_series[normalized_series <= var_95]) if var_95 != 0 else 0
            es_99 = np.mean(normalized_series[normalized_series <= var_99]) if var_99 != 0 else 0

            # Handle potential negative values
            return {
                'var_95': abs(var_95),
                'var_99': abs(var_99),
                'es_95': abs(es_95),
                'es_99': abs(es_99)
            }

        except Exception as e:
            logger.error(f"Error calculating EVT metrics: {e}")
            return {
                'var_95': 0,
                'var_99': 0,
                'es_95': 0,
                'es_99': 0
            }

    def run_statistical_tests(self, symbol: str, options_data: Dict) -> Dict:
        """Run comprehensive statistical tests with enhanced error handling"""
        if self.profiling_enabled:
            self.profile.enable()

        start_time = time.time()

        try:
            iv_series = self._extract_iv_series(options_data)
            if len(iv_series) < 2:
                logger.warning(f"Insufficient data for statistical tests on {symbol}")
                return {}

            # Remove NaN values and ensure sufficient data
            iv_series = iv_series[~np.isnan(iv_series)]
            if len(iv_series) < 2:
                logger.warning(f"Insufficient valid data points after NaN removal for {symbol}")
                return {}

            # Extended Normality Tests with error handling
            try:
                _, shapiro_p = stats.shapiro(iv_series)
                _, ks_p = stats.kstest(iv_series, 'norm')
                stat, crit, sig = anderson(iv_series)
                jb_stat, jb_p = jarque_bera(iv_series)
            except Exception as e:
                logger.error(f"Error in normality tests for {symbol}: {e}")
                shapiro_p = ks_p = jb_p = 1.0
                stat = crit = 0.0
                jb_stat = 0.0

            # Extended Stationarity Tests with error handling
            try:
                adf_stat, adf_p, _ = self._adf_test(iv_series)
                # Handle KPSS test warnings gracefully
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    kpss_stat, kpss_p, _, _ = kpss(iv_series, regression='c', nlags='auto')
            except Exception as e:
                logger.error(f"Error in stationarity tests for {symbol}: {e}")
                adf_stat = kpss_stat = 0.0
                adf_p = kpss_p = 1.0

            # Enhanced Extreme Value Analysis
            z_scores = stats.zscore(iv_series)
            extremes = np.abs(z_scores) > 2
            evt_metrics = self._calculate_evt_metrics(iv_series)

            # Heteroscedasticity Test with error handling
            try:
                lm_stat, lm_p, f_stat, f_p = het_arch(iv_series)
            except Exception as e:
                logger.error(f"Error in heteroscedasticity test for {symbol}: {e}")
                lm_stat = f_stat = 0.0
                lm_p = f_p = 1.0

            # Autocorrelation Test with improved handling
            acf = self._autocorrelation(iv_series)

            result = {
                'normality': {
                    'shapiro_p': shapiro_p,
                    'ks_p': ks_p,
                    'anderson_stat': stat,
                    'anderson_critical': crit,
                    'jarque_bera_stat': jb_stat,
                    'jarque_bera_p': jb_p
                },
                'stationarity': {
                    'adf_stat': adf_stat,
                    'adf_p': adf_p,
                    'kpss_stat': kpss_stat,
                    'kpss_p': kpss_p
                },
                'heteroscedasticity': {
                    'arch_lm_stat': lm_stat,
                    'arch_lm_p': lm_p,
                    'arch_f_stat': f_stat,
                    'arch_f_p': f_p
                },
                'extremes': {
                    'extreme_count': np.sum(extremes),
                    'extreme_ratio': np.mean(extremes),
                    **evt_metrics
                },
                'autocorrelation': {
                    'lag1': acf[1] if len(acf) > 1 else 0,
                    'lag5': acf[5] if len(acf) > 5 else 0
                }
            }

            execution_time = time.time() - start_time
            logger.debug(f"Statistical tests for {symbol} completed in {execution_time:.4f}s")

            if self.profiling_enabled:
                self.profile.disable()

            return result

        except Exception as e:
            logger.error(f"Error running statistical tests for {symbol}: {e}")
            if self.profiling_enabled:
                self.profile.disable()
            return {}

    def _adf_test(self, series: np.array) -> Tuple[float, float, Dict]:
        """Augmented Dickey-Fuller test for stationarity"""
        result = adfuller(series)
        return result[0], result[1], result[4]

    def _autocorrelation(self, series: np.array, nlags: int = 10) -> np.array:
        """Calculate autocorrelation function with improved error handling"""
        try:
            if len(series) <= nlags + 1:
                logger.warning(f"Series length {len(series)} is too short for {nlags} lags")
                return np.array([1.0] + [0.0] * min(nlags, len(series) - 1))

            # Remove NaN values
            series = series[~np.isnan(series)]

            # Calculate autocorrelations with error handling
            acf = [1.0]  # lag 0 autocorrelation is always 1
            for i in range(1, min(len(series), nlags + 1)):
                if len(series) <= i:
                    acf.append(0.0)
                    continue

                # Calculate correlation coefficient safely
                try:
                    corr_matrix = np.corrcoef(series[:-i], series[i:])
                    if corr_matrix.size >= 4:  # 2x2 matrix
                        acf.append(corr_matrix[0, 1])
                    else:
                        acf.append(0.0)
                except (ValueError, RuntimeWarning) as e:
                    logger.warning(f"Error calculating autocorrelation at lag {i}: {e}")
                    acf.append(0.0)

            return np.array(acf)

        except Exception as e:
            logger.error(f"Error in autocorrelation calculation: {e}")
            return np.array([1.0] + [0.0] * nlags)

    def _extract_iv_series(self, options_data: Dict) -> np.array:
        """Extract IV series from options data"""
        iv_values = []
        for expiry in options_data:
            calls = options_data[expiry]['calls']
            puts = options_data[expiry]['puts']
            iv_values.extend(calls['impliedVolatility'].dropna())
            iv_values.extend(puts['impliedVolatility'].dropna())
        return np.array(iv_values)

    def calculate_iv_crush_probability_enhanced(self, symbol: str) -> Dict:
        """Enhanced probability calculation with statistical tests"""
        base_prob = self.calculate_iv_crush_probability(symbol)
        if not base_prob:
            return {}

        options_data = self.fetch_options_data(symbol)
        stat_tests = self.run_statistical_tests(symbol, options_data)

        # Adjust probability based on statistical tests
        prob_adjustments = self._calculate_probability_adjustments(stat_tests)

        final_prob = base_prob['iv_crush_probability'] * (1 + prob_adjustments)
        final_prob = min(max(final_prob, 0), 1)  # Ensure probability is between 0 and 1

        # Include both base metrics and enhanced statistical metrics
        result = {
            'symbol': symbol,
            'iv_crush_probability': final_prob,
            'detail_scores': base_prob['detail_scores'],  # Keep original scores
            'statistical_scores': {  # Add new statistical scores
                'normality_score': 1.0 - stat_tests['normality']['shapiro_p'],
                'stationarity_score': 1.0 - stat_tests['stationarity']['adf_p'],
                'extreme_value_score': stat_tests['extremes']['extreme_ratio'],
                'autocorrelation_score': abs(stat_tests['autocorrelation']['lag1'])
            },
            'statistical_tests': stat_tests,
            'probability_adjustments': prob_adjustments
        }

        # Merge base and statistical scores
        result['detail_scores'].update(result['statistical_scores'])
        return result

    def _calculate_probability_adjustments(self, stat_tests: Dict) -> float:
        """Calculate probability adjustments based on statistical tests and EVT metrics.

        This method implements a comprehensive scoring system that considers multiple
        statistical indicators to adjust the base IV crush probability. The scoring
        system uses the following components and weights:

        Weights:
        - Normality (20%): How well the IV distribution fits a normal distribution
        - Stationarity (20%): Whether the IV series shows mean-reverting behavior
        - Heteroscedasticity (15%): Presence of volatility clustering
        - Extremes (30%): Tail risk and extreme value behavior
        - Autocorrelation (15%): Serial correlation in IV changes

        Adjustment Logic:
        1. Normality:
           - Uses both Shapiro-Wilk and Jarque-Bera tests
           - Higher adjustments for significant non-normality (p < 0.05)

        2. Stationarity:
           - Combines ADF and KPSS tests for robust stationarity assessment
           - Higher adjustments for strong mean-reversion signals

        3. Heteroscedasticity:
           - Uses ARCH-LM test to detect volatility clustering
           - Significant ARCH effects increase crush probability

        4. Extreme Values:
           - Considers both traditional outliers and EVT metrics
           - VaR and ES measures contribute to tail risk assessment
           - Higher adjustments for significant tail risks

        5. Autocorrelation:
           - Examines short-term (lag1) and medium-term (lag5) correlations
           - Strong autocorrelation suggests persistent patterns

        Returns:
            float: The total probability adjustment factor (range: [0, 1])
        """
        adjustments = 0
        weights = {
            'normality': 0.20,
            'stationarity': 0.20,
            'heteroscedasticity': 0.15,
            'extremes': 0.30,  # Increased weight for extreme value analysis
            'autocorrelation': 0.15
        }

        # 1. Normality Adjustments
        if stat_tests.get('normality'):
            shapiro_adj = 0.15 if stat_tests['normality']['shapiro_p'] < 0.05 else 0
            jb_adj = 0.10 if stat_tests['normality']['jarque_bera_p'] < 0.05 else 0
            adjustments += (shapiro_adj + jb_adj) * weights['normality']

        # 2. Stationarity Adjustments
        if stat_tests.get('stationarity'):
            adf_adj = 0.15 if stat_tests['stationarity']['adf_p'] < 0.05 else 0
            kpss_adj = 0.10 if stat_tests['stationarity']['kpss_p'] < 0.05 else 0
            adjustments += (adf_adj + kpss_adj) * weights['stationarity']

        # 3. Heteroscedasticity Adjustments
        if stat_tests.get('heteroscedasticity'):
            arch_adj = 0.20 if stat_tests['heteroscedasticity']['arch_lm_p'] < 0.05 else 0
            adjustments += arch_adj * weights['heteroscedasticity']

        # 4. Enhanced Extreme Values Adjustments
        if stat_tests.get('extremes'):
            extreme_ratio = stat_tests['extremes']['extreme_ratio']
            var_99 = stat_tests['extremes']['var_99']
            es_99 = stat_tests['extremes']['es_99']

            # Assess tail risk using EVT metrics
            tail_risk_adj = 0
            if es_99 < var_99 * 1.5:  # Significant tail risk
                tail_risk_adj += 0.20
            if extreme_ratio > 0.15:  # More than 15% extreme values
                tail_risk_adj += 0.15
            elif extreme_ratio > 0.10:  # More than 10% extreme values
                tail_risk_adj += 0.10

            adjustments += tail_risk_adj * weights['extremes']

        # 5. Autocorrelation Adjustments
        if stat_tests.get('autocorrelation'):
            lag1_acf = abs(stat_tests['autocorrelation']['lag1'])
            if lag1_acf > 0.5:  # Strong autocorrelation
                adjustments += 0.20 * weights['autocorrelation']
            elif lag1_acf > 0.3:  # Moderate autocorrelation
                adjustments += 0.10 * weights['autocorrelation']

        return adjustments

    def visualize_top_candidates(self, n_stocks: int = 10):
        """
        Create visualizations for top IV crush candidates
        """
        results = []
        for symbol in self.sp500_symbols[:30]:  # Analyze first 30 to find top 10
            result = self.calculate_iv_crush_probability_enhanced(symbol)
            if result:
                results.append(result)

        df = pd.DataFrame(results)
        top_10 = df.nlargest(n_stocks, 'iv_crush_probability')

        self._create_probability_distribution_plot(top_10)
        self._create_component_scores_plot(top_10)
        self._create_statistical_tests_plot(top_10)

        return top_10

    def _create_probability_distribution_plot(self, df: pd.DataFrame):
        """Create probability distribution plot"""
        plt.figure(figsize=(12, 6))

        # Probability distribution
        sns.kdeplot(data=df, x='iv_crush_probability', fill=True)

        # Add individual points
        plt.scatter(df['iv_crush_probability'], 
                   np.zeros_like(df['iv_crush_probability']),
                   c='red', alpha=0.5)

        # Add labels
        for _, row in df.iterrows():
            plt.annotate(row['symbol'], 
                        (row['iv_crush_probability'], 0.1),
                        xytext=(0, 5), 
                        textcoords='offset points',
                        ha='center')

        plt.title('IV Crush Probability Distribution for Top Candidates')
        plt.xlabel('Probability')
        plt.ylabel('Density')
        plt.grid(True, alpha=0.3)
        plt.savefig('probability_distribution.png')
        plt.close()

    def _create_component_scores_plot(self, df: pd.DataFrame):
        """Create component scores comparison plot with enhanced metrics"""
        plt.figure(figsize=(15, 8))

        try:
            # Extract and combine all scores
            scores_data = []
            for _, row in df.iterrows():
                if isinstance(row.get('detail_scores'), dict):
                    scores_data.append({
                        'symbol': row['symbol'],
                        **row['detail_scores']
                    })

            if scores_data:
                scores_df = pd.DataFrame(scores_data)
                if not scores_df.empty and len(scores_df.columns) > 1:
                    scores_df.set_index('symbol', inplace=True)

                    # Create heatmap
                    sns.heatmap(scores_df, annot=True, cmap='YlOrRd', center=0.5, fmt='.2f')
                    plt.title('Component Scores Comparison (Including Statistical Metrics)')
                    plt.xlabel('Score Components')
                    plt.ylabel('Symbols')
                    plt.xticks(rotation=45)
                    plt.tight_layout()
                    plt.savefig('component_scores.png')
                else:
                    logger.warning("Insufficient data for component scores visualization")
            else:
                logger.warning("No score data available for visualization")
        except Exception as e:
            logger.error(f"Error creating component scores plot: {e}")
            logger.exception("Full component scores plot error traceback:")
        finally:
            plt.close()

    def _create_statistical_tests_plot(self, df: pd.DataFrame):
        """Create enhanced statistical tests visualization with EVT metrics"""
        plt.figure(figsize=(15, 10))

        # Extract relevant statistical metrics including EVT
        stats_data = []
        for _, row in df.iterrows():
            if 'statistical_tests' in row:
                stats = row['statistical_tests']
                stats_data.append({
                    'symbol': row['symbol'],
                    'Normality (p)': stats['normality']['shapiro_p'],
                    'Stationarity (p)': stats['stationarity']['adf_p'],
                    'ARCH-LM (p)': stats['heteroscedasticity']['arch_lm_p'],
                    'VaR 99%': stats['extremes']['var_99'],
                    'ES 99%': stats['extremes']['es_99'],
                    'ACF(1)': stats['autocorrelation']['lag1']
                })

        stats_df = pd.DataFrame(stats_data)
        if not stats_df.empty:
            stats_df = stats_df.set_index('symbol')

            # Create enhanced heatmap
            sns.heatmap(stats_df, annot=True, cmap='coolwarm', center=0.5, fmt='.3f',
                       cbar_kws={'label': 'Standardized Values'})
            plt.title('Statistical Tests Results (Including EVT Metrics)')
            plt.xlabel('Statistical Metrics')
            plt.ylabel('Symbols')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig('statistical_tests.png')
        plt.close()

def main():
    logger.info("Starting IV Crush Analysis...")
    analyzer = IVCrushAnalyzerEnhanced()

    # Enable profiling (optional)
    # analyzer.enable_profiling()

    logger.info("Analyzing S&P 500 stocks for IV crush probability...")
    top_candidates = analyzer.visualize_top_candidates(10)

    # Disable profiling (optional) and print stats
    #analyzer.disable_profiling()

    logger.info("\nTop 10 IV Crush Candidates:")
    for _, row in top_candidates.iterrows():
        logger.info(f"\nSymbol: {row['symbol']}")
        logger.info(f"IV Crush Probability: {row['iv_crush_probability']:.2%}")
        logger.info("Statistical Tests:")
        for test_name, test_results in row['statistical_tests'].items():
            logger.info(f"  {test_name}:")
            for metric, value in test_results.items():
                # Handle numpy array values
                if isinstance(value, np.ndarray):
                    formatted_value = ", ".join(f"{v:.4f}" for v in value)
                    logger.info(f"    {metric}: [{formatted_value}]")
                else:
                    try:
                        logger.info(f"    {metric}: {float(value):.4f}")
                    except (TypeError, ValueError):
                        logger.info(f"    {metric}: {value}")

if __name__ == "__main__":
    main()

2025-02-13 16:20:17,047 - INFO - Starting IV Crush Analysis...
2025-02-13 16:20:18,667 - INFO - Initialized analyzer with 503 symbols
2025-02-13 16:20:18,669 - INFO - Analyzing S&P 500 stocks for IV crush probability...
2025-02-13 16:20:18,671 - INFO - 
Analyzing MMM...
2025-02-13 16:20:18,673 - INFO - Fetching data for MMM
2025-02-13 16:20:20,336 - INFO - Fetching 2025-02-14 options for MMM
2025-02-13 16:20:20,541 - INFO - Successfully fetched 2025-02-14 options for MMM
2025-02-13 16:20:20,543 - INFO - Fetching 2025-02-21 options for MMM
2025-02-13 16:20:20,749 - INFO - Successfully fetched 2025-02-21 options for MMM
2025-02-13 16:20:20,751 - INFO - Fetching 2025-02-28 options for MMM
2025-02-13 16:20:20,960 - INFO - Successfully fetched 2025-02-28 options for MMM
2025-02-13 16:20:20,962 - INFO - Fetching 2025-03-07 options for MMM
2025-02-13 16:20:21,181 - INFO - Successfully fetched 2025-03-07 options for MMM
2025-02-13 16:20:21,183 - INFO - Cached data for MMM
2025-02-13 16:20:21,8